In [1]:
import tensorflow as tf

print(f"TensorFlow version: {tf.__version__}")
print(f"Built with CUDA: {tf.test.is_built_with_cuda()}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

# Test GPU usage
if tf.config.list_physical_devices('GPU'):
    print("✓ GPU detected and available")
    with tf.device('/GPU:0'):
        a = tf.constant([1.0, 2.0, 3.0])
        b = tf.constant([4.0, 5.0, 6.0])
        c = tf.add(a, b)
        print(f"GPU computation result: {c}")
else:
    print("❌ No GPU available - using CPU")

TensorFlow version: 2.10.1
Built with CUDA: True
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✓ GPU detected and available
GPU computation result: [5. 7. 9.]


## Housekeeping Matters 

In [2]:
'''
This ipynb file is adapted from the previous ResNet training script
Author: Jason Niow
DtaeL 
'''

# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import pathlib

print(f'tensorflow version: {tf.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'numpy version: {np.__version__}')
print(f'seaborn version: {sns.__version__}')

# check tensorflow GPU device support
if len(tf.config.list_physical_devices('GPU')) > 0:
    print('GPU present')
else:
    print('GPU absent')

# paths to load datasets from
train_store_path = 'C:/Users/UserAdmin/Desktop/Jason - Signal Classification/AI Models/Data/synthetic/synthetic_set3_train'
test_store_path = 'C:/Users/UserAdmin/Desktop/Jason - Signal Classification/AI Models/Data/synthetic/synthetic_set3_test'

# convert to pathlib Path objects
train_dir = pathlib.Path(train_store_path)
test_dir = pathlib.Path(test_store_path)

# get list of datasets paths in dir
train_ds_paths = sorted(list(train_dir.glob('*.csv')))
test_ds_paths = sorted(list(test_dir.glob('*.csv')))


# extract classification target from file names
train_ds_type = np.array([x.parts[-1].split('_')[:2] for x in train_ds_paths])
test_ds_type = np.array([x.parts[-1].split('_')[:2] for x in test_ds_paths])

# Get list of classification labels of dataset e.g. 8CPSK, FM, 16qam
train_ds_mod = [s.upper() for s in train_ds_type[:,0]]
test_ds_mod = [s.upper() for s in test_ds_type[:,0]]

# Get list of classification frequency
train_ds_freq = [s.upper() for s in train_ds_type[:, 1]]
test_ds_freq = [s.upper() for s in test_ds_type[:, 1]]

# generate signal type tags
known_signal_tags = {'16QAM', '8CPSK', 'FM'}
signal_tags = {'16QAM': 0, '8CPSK': 1, 'FM': 2, 'UNKNOWN': 3}
# signal_tags = {k : i for i, k in enumerate(np.unique(sorted([s.upper() for s in train_ds_mod] + ['UNKNOWN'])))}

print(signal_tags)


tensorflow version: 2.10.1
pandas version: 2.0.3
numpy version: 1.25.1
seaborn version: 0.13.2
GPU present
{'16QAM': 0, '8CPSK': 1, 'FM': 2, 'UNKNOWN': 3}


In [3]:
for index, s in enumerate(train_ds_type[:,1]):
    if s.upper() == 'FM':
        print(train_ds_type[index, :])

['qpsk' 'fm']
['qpsk' 'fm']
['qpsk' 'fm']
['qpsk' 'fm']
['qpsk' 'fm']
['qpsk' 'fm']


In [4]:
print(set(train_ds_freq))
print(set(train_ds_mod))

{'F70M', 'FM', 'F55M', 'F270M', 'F237M', 'F75M'}
{'NOISE', 'QPSK', 'FM', '16QAM', 'BNET', 'THALES', '8PSK', '64QAM', '8CPSK'}


## Loading Data

In [5]:

# load the dataset(s)

# load dataset information
specs = []
datasets = []

for dataset_paths in [train_ds_paths, test_ds_paths]:
    temp_ds = []
    temp_specs = []

    for path in dataset_paths:
        print(f'loading {path}...', end=' ')

        # load dataset details - Sampling frequency, Number of Samples, Number of Records
        df_spec = pd.read_csv(path, nrows=10, header=None, index_col=0, names=['info'])
        df_spec = df_spec.drop(['Version', 'DateTime', 'TimestampOffset', 'TriggerPosition', 'FastFrameID', 'IDInFastFrame', 'TotalInFastFrame'], axis=0).astype('int')

        temp_specs.append(df_spec)

        # load data, strip unnecessary bits out - I/Q data
        df = pd.read_csv(path, skiprows=10, names=['I', 'Q'])

        df = df.loc[~df['I'].isin(['TimestampOffset', 'TriggerPosition', 'FastFrameID', 'IDInFastFrame', 'TotalInFastFrame'])]
        df['I'] = df['I'].astype('float')

        print(f'loaded')

        temp_ds.append(df)

    datasets.append(temp_ds)
    specs.append(temp_specs)

print('done.')
        


loading C:\Users\UserAdmin\Desktop\Jason - Signal Classification\AI Models\Data\synthetic\synthetic_set3_train\16qam_f237M_bnet_test-2024.08.19.12.41.56.619_chunk_1.csv... loaded
loading C:\Users\UserAdmin\Desktop\Jason - Signal Classification\AI Models\Data\synthetic\synthetic_set3_train\16qam_f237M_bnet_test-2024.08.19.12.42.02.745_chunk_1.csv... loaded
loading C:\Users\UserAdmin\Desktop\Jason - Signal Classification\AI Models\Data\synthetic\synthetic_set3_train\16qam_f237M_bnet_test-2024.08.19.12.42.12.097_chunk_1.csv... loaded
loading C:\Users\UserAdmin\Desktop\Jason - Signal Classification\AI Models\Data\synthetic\synthetic_set3_train\16qam_f237M_bnet_test-2024.08.19.12.42.15.700_chunk_1.csv... loaded
loading C:\Users\UserAdmin\Desktop\Jason - Signal Classification\AI Models\Data\synthetic\synthetic_set3_train\16qam_f237M_bnet_test-2024.08.19.12.42.16.958_chunk_1.csv... loaded
loading C:\Users\UserAdmin\Desktop\Jason - Signal Classification\AI Models\Data\synthetic\synthetic_set3_

In [8]:
# len(specs[0]), len(specs[1])
len(datasets[0]), len(datasets[1])

(9957, 2490)

# Processing Data

In [9]:

# split dataset(s) into records, extract test dataset
processed = []

# number of test records to extract
ntest = 100
rlength = 1024
nrecords = 1
nsamples = 10000

for h, dataset in enumerate(datasets): # loops through training, then testing
    if h == 0:
        dataset_type = 'TRAINING'
    else: dataset_type = 'TESTING'
    temp_processed = []
    specs_df = specs[h]

    print(f'\nType\t\tLocation\tTotal Records\tSamples/Record')
    # Loops through each data point in the dataset
    for i in range(len(dataset)):
        # nrecords = specs_df[i].loc['NumberRecords']['info'] if dataset_type == 'TRAINING' else 400 ### wtf is this
        # nrecords = specs_df[i].loc['NumberRecords']['info']
        # nsamples = specs_df[i].loc['NumberSamples']['info']

        ds_length = dataset[i].shape[0]

        # make life easier
        ds_mod = train_ds_mod if dataset_type == 'TRAINING' else test_ds_mod
        ds_freq = train_ds_freq if dataset_type == 'TRAINING' else test_ds_freq

        # sanity check
        print(f'{ds_mod[i]:<13}\t{ds_freq[i]:<15}\t{nrecords:<7}\t\t{nsamples:<7}')

        # loop through dataset to split
        for j in range(nrecords):
            # extract sample length worth of samples for each record, then transpose for easier access later
            record = dataset[i].iloc[(nsamples * j):(nsamples * (j+1))].values.T

            # pad shorter records with random padding to rlength
            if nsamples < rlength:
                print(f"i: {i} j : {j} Sample length {nsamples} is lesser than {rlength}")
                # deterine pad amount
                pad_length = rlength - nsamples
                lpad_length = np.random.randint(0, pad_length+1)
                rpad_length = pad_length - lpad_length

                # generate pad
                lpad = np.zeros((2, lpad_length))
                rpad = np.zeros((2, rpad_length))

                # concatenate pad
                record = np.concatenate([lpad, record, rpad], axis=1)

            # truncate longer records to rlength
            elif nsamples > rlength:
                # print(f"i: {i} j : {j} Sample length {nsamples} is greater than {rlength}")
                record = record[:,:rlength]

            # add processed record to list
            signal_tag = signal_tags.get(ds_mod[i], signal_tags['UNKNOWN']) # default to UNKNOWN if not one of the known classes
            temp_processed.append([ds_mod[i], signal_tag, ds_freq[i], record])

    processed.append(temp_processed)

# convert list into dataframes for later use, randomise, extract test records
df_train = pd.DataFrame(processed[0], columns=['signal_type', 'tag', 'location', 'record']).sample(frac=1, random_state=42)
df_test = pd.DataFrame(processed[1], columns=['signal_type', 'tag', 'location', 'record']).sample(frac=1, random_state=42)

# print dataset statistics
print(f'\n{"Stats":^30}')
print(f'Dataset\tLength\tRecords/Sample')
print(f'Train\t{df_train.shape[0]:<5}\t{df_train["record"].iloc[0].shape[1]}')
print(f'Test\t{df_test.shape[0]:<5}\t{df_train["record"].iloc[0].shape[1]}')


# define one hot encode function
def one_hot(arr, n_cat):
    output = []
    for n in arr:
        result = np.zeros(n_cat)
        result[n] = 1

        output.append(result)

    return np.array(output, dtype=int)

# extract train and test data
X_train = np.concatenate(df_train['record'].values).reshape((df_train.shape[0], 2, rlength, 1))
y_train = one_hot(df_train['tag'].values, len(signal_tags))

#X_test = np.concatenate(df_test['record'].values).reshape((df_test.shape[0], 2, rlength, 1))
#y_test = one_hot(df_test['tag'].values, len(signal_tags))



Type		Location	Total Records	Samples/Record
16QAM        	F237M          	1      		10000  
16QAM        	F237M          	1      		10000  
16QAM        	F237M          	1      		10000  
16QAM        	F237M          	1      		10000  
16QAM        	F237M          	1      		10000  
16QAM        	F237M          	1      		10000  
16QAM        	F237M          	1      		10000  
16QAM        	F237M          	1      		10000  
16QAM        	F237M          	1      		10000  
16QAM        	F237M          	1      		10000  
16QAM        	F237M          	1      		10000  
16QAM        	F237M          	1      		10000  
16QAM        	F237M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F

## Creating Model (without dropout)

In [15]:

# import model stuff
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, concatenate, Dense, Input, Flatten

# functions for model segments
def res_unit(x, dim, n):
    '''
    function that creates a residual unit for each residual stack.

    INPUT PARAMETERS
    x: layer to connect to
    dim: size of layer
    n: number of units to create
    '''

    for _ in range(n):
        u = Conv2D(dim, 2, activation='relu', padding='same')(x)
        u = Conv2D(dim, 2, activation='linear', padding='same')(u)

        # skip-con
        x = concatenate([u, x])

    return x

def res_stack(x, dim):
    '''
    function that creates a residual stack for the model

    INPUT PARAMETERS
    x: layer to connect to
    dim: size of stack
    '''

    s = Conv2D(dim, 1, activation='linear', padding='same')(x)
    s = res_unit(s, dim, 2)
    s = MaxPooling2D(2, padding='same')(s)

    return s

# function to create main model
def create_model(in_dim, out_dim):
    '''
    function to construct the actual resnet model.

    INPUT PARAMETERS
    in_dim: dimensions of input
    out_dim: size of output
    '''

    input_layer = Input(in_dim)

    # res stacks
    x = res_stack(input_layer, 512)
    x = res_stack(x, 256)
    x = res_stack(x, 128)
    x = res_stack(x, 64)
    x = res_stack(x, 32)
    x = res_stack(x, 16)

    # fully connected layers
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(128, activation='relu')(x)

    output_layer = Dense(out_dim, activation='softmax')(x)

    # turn layers into model
    model = Model(inputs=input_layer, outputs=output_layer, name='resnet_rf_classification_model')

    return model

# model = create_model((2, rlength, 1), len(signal_tags))


# Bayesian Optimization for Hyperparameter Tuning

In this section, we'll use Bayesian Optimization to automatically find the best hyperparameters for our ResNet model. This includes optimizing learning rate, batch size, epochs, dropout rates, and dense layer sizes.

In [10]:
# Install required packages for Bayesian Optimization
import subprocess
import sys
import skopt

# Import Bayesian Optimization libraries
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt.plots import plot_convergence, plot_objective
import time
import json
from datetime import datetime

In [11]:
# Define hyperparameter search space
dimensions = [
    Real(low=1e-5, high=1e-2, prior='log-uniform', name='learning_rate'),
    Integer(low=16, high=128, name='batch_size'),
    Integer(low=10, high=100, name='epochs'),
    Real(low=0.0, high=0.5, name='dropout_rate'),
    Integer(low=64, high=512, name='dense_units'),
    Integer(low=5, high=20, name='patience')
]

# Extract dimension names for easy reference
dimension_names = [dim.name for dim in dimensions]
print("Hyperparameter search space:")
for dim in dimensions:
    print(f"  {dim.name}: {dim}")

# Store results for analysis
optimization_results = []

Hyperparameter search space:
  learning_rate: Real(low=1e-05, high=0.01, prior='log-uniform', transform='identity')
  batch_size: Integer(low=16, high=128, prior='uniform', transform='identity')
  epochs: Integer(low=10, high=100, prior='uniform', transform='identity')
  dropout_rate: Real(low=0.0, high=0.5, prior='uniform', transform='identity')
  dense_units: Integer(low=64, high=512, prior='uniform', transform='identity')
  patience: Integer(low=5, high=20, prior='uniform', transform='identity')


In [16]:
# Enhanced model creation function with hyperparameters
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, concatenate, Dense, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def create_optimized_model(in_dim, out_dim, dropout_rate=0.2, dense_units=128):
    """
    Create ResNet model with configurable hyperparameters
    
    Args:
        in_dim: Input dimensions
        out_dim: Output dimensions  
        dropout_rate: Dropout rate for regularization
        dense_units: Number of units in dense layers
    """
    input_layer = Input(in_dim)

    # res stacks (same as original)
    x = res_stack(input_layer, 512)
    x = res_stack(x, 256)
    x = res_stack(x, 128)
    x = res_stack(x, 64)
    x = res_stack(x, 32)
    x = res_stack(x, 16)

    # fully connected layers with dropout
    x = Flatten()(x)
    x = Dense(dense_units, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(dense_units, activation='relu')(x)
    x = Dropout(dropout_rate)(x)

    output_layer = Dense(out_dim, activation='softmax')(x)

    # turn layers into model
    model = Model(inputs=input_layer, outputs=output_layer, name='optimized_resnet_rf_classification_model')

    return model

print("Enhanced model creation function defined with dropout and configurable dense units")

Enhanced model creation function defined with dropout and configurable dense units


In [17]:
# Define the objective function for Bayesian Optimization
@use_named_args(dimensions)
def objective_function(**params):
    """
    Objective function to minimize (validation loss)
    
    Returns the validation loss after training with given hyperparameters
    """
    print(f"\n{'='*60}")
    print(f"Training with hyperparameters: {params}")
    print(f"{'='*60}")
    
    try:
        # Clear any existing models from memory
        tf.keras.backend.clear_session()
        
        # Create model with current hyperparameters
        model = create_optimized_model(
            in_dim=(2, rlength, 1), 
            out_dim=len(signal_tags),
            dropout_rate=params['dropout_rate'],
            dense_units=params['dense_units']
        )
        
        # Compile model
        optimizer = Adam(learning_rate=params['learning_rate'])
        model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # Create early stopping callback
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=params['patience'],
            verbose=1,
            mode='min',
            restore_best_weights=True
        )
        
        # Train model
        start_time = time.time()
        history = model.fit(
            X_train, y_train,
            epochs=params['epochs'],
            batch_size=params['batch_size'],
            validation_split=0.2,
            callbacks=[early_stopping],
            verbose=1
        )
        
        training_time = time.time() - start_time
        
        # Get best validation loss
        val_loss = min(history.history['val_loss'])
        val_accuracy = max(history.history['val_accuracy'])
        
        # Store results
        result = {
            'params': params,
            'val_loss': val_loss,
            'val_accuracy': val_accuracy,
            'training_time': training_time,
            'epochs_trained': len(history.history['loss']),
            'timestamp': datetime.now().isoformat()
        }
        optimization_results.append(result)
        
        print(f"\\nResult: Val Loss = {val_loss:.4f}, Val Accuracy = {val_accuracy:.4f}")
        print(f"Training time: {training_time:.2f} seconds")
        print(f"Epochs trained: {len(history.history['loss'])}")
        
        return val_loss
        
    except Exception as e:
        print(f"Error during training: {str(e)}")
        # Return a high loss value for failed trials
        return 10.0

print("Objective function defined successfully")

Objective function defined successfully


## Custom Initial Hyperparameters

Define your preferred starting hyperparameters for the Bayesian optimization. These will be evaluated first before the algorithm begins its intelligent search.

In [ ]:
# Define your preferred initial hyperparameters
# You can modify these based on your domain knowledge or previous experiments

initial_hyperparameters = [
    # Hyperparameter set 1: Your current baseline/default
    {
        'learning_rate': 0.0003,
        'batch_size': 32,
        'epochs': 50,
        'dropout_rate': 0.2,
        'dense_units': 128,
        'patience': 10
    },
    
    # Hyperparameter set 2: Lower learning rate, higher regularization
    {
        'learning_rate': 0.0001,
        'batch_size': 64,
        'epochs': 80,
        'dropout_rate': 0.3,
        'dense_units': 256,
        'patience': 15
    },
    
    # Hyperparameter set 3: Higher learning rate, less regularization
    {
        'learning_rate': 0.001,
        'batch_size': 16,
        'epochs': 40,
        'dropout_rate': 0.1,
        'dense_units': 512,
        'patience': 8
    },
    
    # Hyperparameter set 4: Conservative balanced approach
    {
        'learning_rate': 0.0005,
        'batch_size': 48,
        'epochs': 60,
        'dropout_rate': 0.25,
        'dense_units': 192,
        'patience': 12
    }
]

print("🎯 INITIAL HYPERPARAMETER SETS DEFINED:")
print("="*60)
for i, params in enumerate(initial_hyperparameters, 1):
    print(f"Set {i}:")
    for param, value in params.items():
        if param == 'learning_rate':
            print(f"  {param}: {value:.1e}")
        else:
            print(f"  {param}: {value}")
    print()

# Convert initial hyperparameters to the format expected by skopt
initial_points = []
for params in initial_hyperparameters:
    point = [params[name] for name in dimension_names]
    initial_points.append(point)

print(f"✓ {len(initial_hyperparameters)} initial hyperparameter sets ready for optimization")
print(f"✓ Initial points converted to skopt format: {len(initial_points)} points")

In [ ]:
# Function to test individual hyperparameter sets before full optimization
def test_hyperparameter_set(params, set_name="Custom"):
    """
    Test a single hyperparameter set and return results
    """
    print(f"\n🧪 TESTING HYPERPARAMETER SET: {set_name}")
    print(f"{'='*50}")
    for param, value in params.items():
        if param == 'learning_rate':
            print(f"  {param}: {value:.1e}")
        else:
            print(f"  {param}: {value}")
    print(f"{'='*50}")
    
    # Convert params to list format for objective function
    param_values = [params[name] for name in dimension_names]
    
    # Test the hyperparameters
    result = objective_function(*param_values)
    
    return result

def test_all_initial_sets():
    """
    Test all predefined initial hyperparameter sets
    """
    print("🚀 TESTING ALL INITIAL HYPERPARAMETER SETS")
    print("="*60)
    
    results_summary = []
    
    for i, params in enumerate(initial_hyperparameters, 1):
        set_name = f"Initial Set {i}"
        result = test_hyperparameter_set(params, set_name)
        
        if result is not None:
            results_summary.append({
                'set_name': set_name,
                'params': params,
                'val_loss': result
            })
    
    # Display summary
    if results_summary:
        print(f"\n📊 INITIAL HYPERPARAMETER SETS SUMMARY:")
        print(f"{'='*80}")
        print(f"{'Set Name':<15} {'Val Loss':<12} {'LR':<12} {'Batch':<8} {'Dropout':<10} {'Dense':<8}")
        print(f"{'-'*80}")
        
        # Sort by validation loss
        results_summary.sort(key=lambda x: x['val_loss'])
        
        for result in results_summary:
            params = result['params']
            print(f"{result['set_name']:<15} {result['val_loss']:<12.4f} "
                  f"{params['learning_rate']:<12.2e} {params['batch_size']:<8} "
                  f"{params['dropout_rate']:<10.3f} {params['dense_units']:<8}")
        
        best_set = results_summary[0]
        print(f"\n🏆 BEST INITIAL SET: {best_set['set_name']} (Val Loss: {best_set['val_loss']:.4f})")
        
        return results_summary
    
    return None

print("\n🔧 TESTING FUNCTIONS AVAILABLE:")
print("1. test_all_initial_sets() - Test all predefined sets")
print("2. test_hyperparameter_set(your_params, 'Your Name') - Test custom set")
print("\nExample usage:")
print("  results = test_all_initial_sets()")
print("  custom_result = test_hyperparameter_set(initial_hyperparameters[0], 'My Test')")

## How to Customize Your Initial Hyperparameters

### 📝 **Editing Initial Hyperparameters**

To use your own starting hyperparameters:

1. **Modify the `initial_hyperparameters` list above** with your preferred values
2. **Add or remove hyperparameter sets** as needed
3. **Ensure all parameters are within the search space bounds** defined in `dimensions`

### 🎯 **Parameter Guidelines**

- **learning_rate**: 1e-5 to 1e-2 (try: 0.0001, 0.0003, 0.001, 0.003)
- **batch_size**: 16 to 128 (try: 16, 32, 48, 64, 96, 128)
- **epochs**: 10 to 100 (try: 30, 50, 80, 100)
- **dropout_rate**: 0.0 to 0.5 (try: 0.1, 0.2, 0.3, 0.4)
- **dense_units**: 64 to 512 (try: 128, 192, 256, 384, 512)
- **patience**: 5 to 20 (try: 8, 10, 12, 15)

### 🚀 **Recommended Workflow**

1. **Test individual sets first**: `test_all_initial_sets()`
2. **Run quick optimization**: `run_quick_optimization()`
3. **Run full optimization**: `run_full_optimization()`
4. **Analyze results** with the visualization cells

In [ ]:
# Run Bayesian Optimization with Custom Initial Points
def run_bayesian_optimization_with_initial_points(n_calls=15, n_random_starts=2, use_initial_points=True):
    """
    Run Bayesian optimization with your custom initial hyperparameters
    
    Args:
        n_calls: Total number of optimization calls
        n_random_starts: Additional random points beyond initial points
        use_initial_points: Whether to use your predefined initial points
    """
    
    print("🚀 STARTING BAYESIAN OPTIMIZATION")
    print("="*70)
    
    if use_initial_points and 'initial_points' in globals():
        print(f"📊 Configuration:")
        print(f"  Total calls: {n_calls}")
        print(f"  Initial points: {len(initial_points)} (your predefined sets)")
        print(f"  Additional random starts: {n_random_starts}")
        print(f"  Guided optimization calls: {n_calls - len(initial_points) - n_random_starts}")
        
        # Clear previous results
        optimization_results.clear()
        
        # Run optimization with your initial points
        optimization_start_time = time.time()
        
        result = gp_minimize(
            func=objective_function,
            dimensions=dimensions,
            n_calls=n_calls,
            n_initial_points=n_random_starts,  # Additional random points
            x0=initial_points,  # Your predefined starting points
            acq_func='EI',  # Expected Improvement acquisition function
            random_state=42
        )
        
    else:
        print(f"📊 Configuration (Standard Mode):")
        print(f"  Total calls: {n_calls}")
        print(f"  Random initial points: {n_random_starts + 2}")
        print(f"  Guided optimization calls: {n_calls - n_random_starts - 2}")
        
        # Clear previous results
        optimization_results.clear()
        
        # Run standard optimization
        optimization_start_time = time.time()
        
        result = gp_minimize(
            func=objective_function,
            dimensions=dimensions,
            n_calls=n_calls,
            n_initial_points=n_random_starts + 2,  # Standard random points
            acq_func='EI',  # Expected Improvement acquisition function
            random_state=42
        )
    
    return result, optimization_start_time

# Quick optimization (fewer calls for testing)
def run_quick_optimization():
    """Run a quick optimization for testing"""
    return run_bayesian_optimization_with_initial_points(n_calls=8, n_random_starts=1)

# Full optimization 
def run_full_optimization():
    """Run full optimization with more calls"""
    return run_bayesian_optimization_with_initial_points(n_calls=20, n_random_starts=3)

# Execute the optimization (you can choose which one to run)
print("🎮 OPTIMIZATION OPTIONS:")
print("1. run_quick_optimization() - Fast test (8 total calls)")
print("2. run_full_optimization() - Thorough search (20 calls)")
print("3. run_bayesian_optimization_with_initial_points(n_calls, n_random_starts) - Custom")
print("\nChoose and run one of the above functions, or run the default below:")
print("="*70)

# Default run (you can modify this)
result, optimization_start_time = run_bayesian_optimization_with_initial_points(n_calls=15, n_random_starts=2)

optimization_total_time = time.time() - optimization_start_time

print(f"\\n{'='*80}")
print(f"🎉 BAYESIAN OPTIMIZATION COMPLETED!")
print(f"{'='*80}")
print(f"⏱️  Total optimization time: {optimization_total_time:.2f} seconds")
print(f"🏆 Best validation loss: {result.fun:.4f}")
print(f"\\n🎯 Best hyperparameters:")
best_params = {}
for name, value in zip(dimension_names, result.x):
    best_params[name] = value
    if name == 'learning_rate':
        print(f"  {name}: {value:.2e}")
    elif name in ['batch_size', 'dense_units', 'epochs', 'patience']:
        print(f"  {name}: {int(value)}")
    else:
        print(f"  {name}: {value:.4f}")

# Analyze initial points performance if they were used
if 'initial_points' in globals() and len(optimization_results) >= len(initial_points):
    print(f"\\n📈 INITIAL POINTS PERFORMANCE:")
    print(f"{'Point':<20} {'Val Loss':<12} {'Type':<15}")
    print(f"{'-'*50}")
    
    for i in range(len(initial_points)):
        if i < len(optimization_results):
            loss = optimization_results[i]['val_loss']
            print(f"Initial Set {i+1:<12} {loss:<12.4f} Predefined")

# Save optimization results
results_dict = {
    'best_params': best_params,
    'best_loss': float(result.fun),
    'all_results': optimization_results,
    'optimization_time': optimization_total_time,
    'n_calls': len(optimization_results),
    'initial_points_used': initial_hyperparameters if 'initial_hyperparameters' in globals() else None
}

# Save to JSON file with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'bayesian_optimization_results_{timestamp}.json'
with open(filename, 'w') as f:
    json.dump(results_dict, f, indent=2)

print(f"\\n💾 Results saved to '{filename}'")

Starting Bayesian Optimization...
This will evaluate multiple hyperparameter combinations to find the optimal settings.
Each iteration will train a model and evaluate its performance.\n

Training with hyperparameters: {'learning_rate': 0.002452612631133679, 'batch_size': 37, 'epochs': 80, 'dropout_rate': 0.29842507897324355, 'dense_units': 264, 'patience': 6}
Epoch 1/80
216/216 [==============================] - 106s 486ms/step - loss: 2.8592 - accuracy: 0.5048 - val_loss: 0.9038 - val_accuracy: 0.5964
Epoch 2/80
216/216 [==============================] - 94s 434ms/step - loss: 0.8751 - accuracy: 0.6093 - val_loss: 0.8049 - val_accuracy: 0.6657
Epoch 3/80
216/216 [==============================] - 94s 435ms/step - loss: 0.8358 - accuracy: 0.6490 - val_loss: 0.7433 - val_accuracy: 0.6817
Epoch 4/80
216/216 [==============================] - 94s 433ms/step - loss: 1.1394 - accuracy: 0.5468 - val_loss: 0.8487 - val_accuracy: 0.6340
Epoch 5/80
216/216 [==============================] - 93s

ValueError: Input y contains NaN.

In [ ]:
# Visualize Optimization Results
import matplotlib.pyplot as plt

# 1. Convergence plot
plt.figure(figsize=(15, 10))

plt.subplot(2, 3, 1)
plot_convergence(result)
plt.title('Bayesian Optimization Convergence')
plt.xlabel('Number of calls')
plt.ylabel('Validation Loss')

# 2. Hyperparameter vs Performance plots
param_names = ['learning_rate', 'batch_size', 'dropout_rate', 'dense_units']
param_indices = [dimension_names.index(name) for name in param_names if name in dimension_names]

for i, (param_idx, param_name) in enumerate(zip(param_indices[:4], param_names[:4])):
    plt.subplot(2, 3, i+2)
    param_values = [res['params'][param_name] for res in optimization_results]
    val_losses = [res['val_loss'] for res in optimization_results]
    
    plt.scatter(param_values, val_losses, alpha=0.7)
    plt.xlabel(param_name)
    plt.ylabel('Validation Loss')
    plt.title(f'{param_name} vs Val Loss')
    
    if param_name == 'learning_rate':
        plt.xscale('log')

# 3. Training time vs Performance
plt.subplot(2, 3, 6)
training_times = [res['training_time'] for res in optimization_results]
val_losses = [res['val_loss'] for res in optimization_results]
plt.scatter(training_times, val_losses, alpha=0.7)
plt.xlabel('Training Time (seconds)')
plt.ylabel('Validation Loss')
plt.title('Training Time vs Val Loss')

plt.tight_layout()
plt.show()

# Print detailed results
print("\\nDetailed Results Summary:")
print("="*80)
sorted_results = sorted(optimization_results, key=lambda x: x['val_loss'])

print(f"{'Rank':<4} {'Val Loss':<10} {'Val Acc':<10} {'LR':<10} {'Batch':<6} {'Dropout':<8} {'Dense':<6}")
print("-" * 80)

for i, res in enumerate(sorted_results[:10]):  # Top 10 results
    params = res['params']
    print(f"{i+1:<4} {res['val_loss']:<10.4f} {res['val_accuracy']:<10.4f} "
          f"{params['learning_rate']:<10.2e} {params['batch_size']:<6} "
          f"{params['dropout_rate']:<8.3f} {params['dense_units']:<6}")

print("\\nBest hyperparameters found:")
best_params = dict(zip(dimension_names, result.x))
for param, value in best_params.items():
    print(f"  {param}: {value}")

print(f"\\nBest validation loss: {result.fun:.4f}")

# Calculate improvement over baseline (if you want to compare with your original hyperparameters)
baseline_loss = max([res['val_loss'] for res in optimization_results])  # Worst case as baseline
improvement = ((baseline_loss - result.fun) / baseline_loss) * 100
print(f"Improvement over worst trial: {improvement:.2f}%")

In [ ]:
# Train Final Model with Optimal Hyperparameters
print("Training final model with optimal hyperparameters...")

# Clear any existing models
tf.keras.backend.clear_session()

# Get best hyperparameters
best_params = dict(zip(dimension_names, result.x))

# Create and compile the optimal model
optimal_model = create_optimized_model(
    in_dim=(2, rlength, 1),
    out_dim=len(signal_tags),
    dropout_rate=best_params['dropout_rate'],
    dense_units=int(best_params['dense_units'])
)

optimal_model.compile(
    optimizer=Adam(learning_rate=best_params['learning_rate']),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train with optimal hyperparameters
final_callback = EarlyStopping(
    monitor='val_loss',
    patience=int(best_params['patience']),
    verbose=1,
    mode='min',
    restore_best_weights=True
)

print(f"Training with optimal hyperparameters:")
for param, value in best_params.items():
    print(f"  {param}: {value}")

final_history = optimal_model.fit(
    X_train, y_train,
    epochs=int(best_params['epochs']),
    batch_size=int(best_params['batch_size']),
    validation_split=0.2,
    callbacks=[final_callback],
    verbose=1
)

# Save the optimal model
optimal_model.save('C:/Users/UserAdmin/Desktop/Jason - Signal Classification/AI Models/trained_models/bayesian_optimized_model')
print("\\nOptimal model saved to 'trained_models/bayesian_optimized_model'")

# Store final training history
final_train_hist = final_history.history

print(f"\\nFinal model performance:")
print(f"Best validation loss: {min(final_train_hist['val_loss']):.4f}")
print(f"Best validation accuracy: {max(final_train_hist['val_accuracy']):.4f}")
print(f"Epochs trained: {len(final_train_hist['loss'])}")

In [ ]:
# Compare Original vs Optimized Model Performance
plt.figure(figsize=(15, 10))

# Plot 1: Training and Validation Loss Comparison
plt.subplot(2, 3, 1)
plt.plot(final_train_hist['loss'], label='Optimized - Training Loss', linewidth=2)
plt.plot(final_train_hist['val_loss'], label='Optimized - Validation Loss', linewidth=2)
plt.title('Optimized Model: Training vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Training and Validation Accuracy Comparison  
plt.subplot(2, 3, 2)
plt.plot(final_train_hist['accuracy'], label='Optimized - Training Accuracy', linewidth=2)
plt.plot(final_train_hist['val_accuracy'], label='Optimized - Validation Accuracy', linewidth=2)
plt.title('Optimized Model: Training vs Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Hyperparameter Importance (based on variance in results)
plt.subplot(2, 3, 3)
param_variances = {}
for param_name in ['learning_rate', 'batch_size', 'dropout_rate', 'dense_units']:
    if param_name in dimension_names:
        param_values = [res['params'][param_name] for res in optimization_results]
        losses = [res['val_loss'] for res in optimization_results]
        
        # Normalize parameters for fair comparison
        if param_name == 'learning_rate':
            param_values = np.log10(param_values)
        
        correlation = np.corrcoef(param_values, losses)[0, 1]
        param_variances[param_name] = abs(correlation)

param_names = list(param_variances.keys())
param_importance = list(param_variances.values())

plt.bar(param_names, param_importance)
plt.title('Hyperparameter Importance\\n(Absolute Correlation with Val Loss)')
plt.ylabel('Importance Score')
plt.xticks(rotation=45)

# Plot 4: Optimization Progress
plt.subplot(2, 3, 4)
cumulative_best = []
current_best = float('inf')
for res in optimization_results:
    if res['val_loss'] < current_best:
        current_best = res['val_loss']
    cumulative_best.append(current_best)

plt.plot(range(1, len(cumulative_best)+1), cumulative_best, 'b-', linewidth=2, marker='o')
plt.title('Optimization Progress\\n(Best Loss Found So Far)')
plt.xlabel('Iteration')
plt.ylabel('Best Validation Loss')
plt.grid(True, alpha=0.3)

# Plot 5: Parameter Distribution of Best Results
plt.subplot(2, 3, 5)
best_results = sorted(optimization_results, key=lambda x: x['val_loss'])[:5]
learning_rates = [res['params']['learning_rate'] for res in best_results]
plt.scatter(range(len(learning_rates)), learning_rates, s=100, alpha=0.7)
plt.yscale('log')
plt.title('Learning Rates of Top 5 Models')
plt.xlabel('Rank (1=best)')
plt.ylabel('Learning Rate')
plt.grid(True, alpha=0.3)

# Plot 6: Training Time vs Performance Trade-off
plt.subplot(2, 3, 6)
training_times = [res['training_time'] for res in optimization_results]
val_accuracies = [res['val_accuracy'] for res in optimization_results]
colors = [res['val_loss'] for res in optimization_results]

scatter = plt.scatter(training_times, val_accuracies, c=colors, cmap='viridis_r', alpha=0.7)
plt.colorbar(scatter, label='Validation Loss')
plt.xlabel('Training Time (seconds)')
plt.ylabel('Validation Accuracy')
plt.title('Training Time vs Accuracy\\n(Color = Val Loss)')

plt.tight_layout()
plt.show()

# Summary Statistics
print("\\n" + "="*80)
print("BAYESIAN OPTIMIZATION SUMMARY")
print("="*80)

print(f"\\nOptimization Configuration:")
print(f"  Number of trials: {len(optimization_results)}")
print(f"  Total optimization time: {optimization_total_time:.2f} seconds")
print(f"  Average time per trial: {optimization_total_time/len(optimization_results):.2f} seconds")

print(f"\\nBest Model Performance:")
best_result = min(optimization_results, key=lambda x: x['val_loss'])
print(f"  Validation Loss: {best_result['val_loss']:.4f}")
print(f"  Validation Accuracy: {best_result['val_accuracy']:.4f}")
print(f"  Training Time: {best_result['training_time']:.2f} seconds")
print(f"  Epochs Trained: {best_result['epochs_trained']}")

print(f"\\nOptimal Hyperparameters:")
for param, value in best_params.items():
    if param == 'learning_rate':
        print(f"  {param}: {value:.2e}")
    elif param in ['batch_size', 'dense_units', 'epochs', 'patience']:
        print(f"  {param}: {int(value)}")
    else:
        print(f"  {param}: {value:.4f}")

print(f"\\nPerformance Distribution:")
val_losses = [res['val_loss'] for res in optimization_results]
print(f"  Best validation loss: {min(val_losses):.4f}")
print(f"  Worst validation loss: {max(val_losses):.4f}")
print(f"  Mean validation loss: {np.mean(val_losses):.4f}")
print(f"  Std validation loss: {np.std(val_losses):.4f}")

improvement = ((max(val_losses) - min(val_losses)) / max(val_losses)) * 100
print(f"  Improvement range: {improvement:.2f}%")